In [2]:
import pulp
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('data/players_cleaned.csv')

In [14]:
player_names = df['name']
player_cost = df['cost']
player_points = df['points']
player_pos = df[['position_DEF', 'position_GKP', 'position_FWD', 'position_MID']]
teams = set(df.columns) - set(['name', 'cost', 'points', 'position_DEF', 'position_GKP', 'position_FWD', 'position_MID'])
player_team = df[list(teams)]

In [20]:
assert(len(player_team.columns) == 20)
assert(len(player_pos.columns) == 4)

In [27]:
player_binary_choice = [pulp.LpVariable('x' + str(i), cat='Binary') for i in range(len(player_names))]

In [31]:
assert(len(player_binary_choice) == len(player_names))

In [44]:
FPL_problem = pulp.LpProblem("Maximize_Fantasy_Points", pulp.LpMaximize)

I will add the objective function to our problem

In [47]:
objective = 0
for i in range(len(player_binary_choice)):
    objective += player_binary_choice[i] * player_cost[i]

FPL_problem += objective, 'Objective'

I will now add the position constraints

In [51]:
number_of = {'position_GKP': 2, 'position_FWD': 3, 'position_MID': 5, 'position_DEF': 5}
for pos in player_pos.columns:
    x = 0
    for i in range(len(player_binary_choice)):
        x += player_binary_choice[i] * player_pos[pos][i]
    FPL_problem += x == number_of[pos], 'Constraint_' + pos

I will now add the team constraint

In [ ]:
I will now add the cost constraint